Парсер резюме с hh.
Копия паресера вакансий, но без сбора контактов.

Парсер доработан 20 октября 2021.

In [ ]:
#Загружаю нужные библиотеки

import pandas as pd
import math

import re

# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# селениум
from selenium import webdriver

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [ ]:
# тестовая страница для Селениума

test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [ ]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
opt.add_argument('--proxy-server=194.67.193.186:25185') # "девственный" прокси(:
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [ ]:
# запускаю драйвер, проверяю настройки

driver = webdriver.Chrome(options=opt)
driver.get(test_url)

In [ ]:
# администратор стоматологии / Медицина, фармацевтика
url_part_1 = 'https://hh.ru/search/resume?area=113&clusters=true&exp_period=all_time&label=only_with_salary&logic=normal&no_magic=true&order_by=relevance&ored_clusters=true&pos=full_text&specialization=13.185&specialization=13.227&specialization=13.433&specialization=13.732&text='

url_part_2 = '&items_on_page=100&page='

resume_queries = ['Администратор+стоматологии', 'Администратор+стоматологической+клиники']

Кол-во страниц, вытаскиваю из блока внизу (переход по ссылкам).

На hh пагинация страниц начинается с нуля, а номер последней страницы всегда перед кнопкой "Дальше".

Чтобы не грузить первую страницу 2 раза, сначала собираю ссылки на ней, а потом перебираю циклом оставшиеся и собираю ссылки тем же скриптом.

Собираю заголовки на первом этапе, поскольку верстка страниц резюме у hh неудобная.

In [ ]:
# собираю ссылки

data = [] # список для данных

for query in tqdm(resume_queries): # цикл для перебора запросов
    driver.get(url_part_1 + query + url_part_2 + '0')
    sleep(3)
      
    page_number = int((driver.find_element_by_class_name('pager')
                       .find_element_by_class_name('bloko-button-group')
                       .text
                       .replace('.', '').replace('\nдальше', '')
                       .split('\n'))[-1]
                     )
    # забираю ссылки с первой страницы 
    all_urls = driver.find_elements_by_css_selector('a.resume-search-item__name') # забираю ссылки

    for url_page in all_urls: # разбираю ссылки и добавляю в список
        
        topic = url_page.text # заголовок
        link = [re.sub('\?query=(.)+', '', url_page.get_attribute('href'))] # сама ссылка
        data.append([topic, link]) # добавляю в список

        
    for i in range(1, page_number):
        driver.get(url_part_1 + query + url_part_2 + str(i))
        sleep(3)
        
        # забираю ссылки тем же кодом
        all_urls = driver.find_elements_by_css_selector('a.resume-search-item__name')
        
        for url_page in all_urls: # разбираю ссылки и добавляю в список
            topic = url_page.text # заголовок
            link = [re.sub('\?query=(.)+', '', url_page.get_attribute('href'))] # сама ссылка
            data.append([topic, link]) # добавляю в список

In [ ]:
# преобразовываю в датафрейм

head = ['topic', 'link']
work_link = pd.DataFrame(data, columns=head)

In [ ]:
work_link.head()

In [ ]:
# проверяю, нет ли повторов

work_link.shape

Руками выкинул "лишние" вакансии.

In [ ]:
#work_link.to_excel('C:/00_Data/Клиники_вакансии/hh_resume_links.xlsx', encoding='utf-8', index=False)

In [ ]:
#work_link = pd.read_excel('C:/00_Data/Клиники_вакансии/hh_resume_links.xlsx', sheet_name='data_2')

Собираю название резюме, ожидаемый размер ЗП, краткую информацию.
Также полный текст резюме (но сейчас он мне не нужен, поэтому не разбираю)

In [ ]:
data = [] # список для данных

for item in tqdm(range(work_link.shape[0])): # цикл для перехода по ссылкам    
    driver.get(work_link.iloc[item].link)
    sleep(3)    
    driver.implicitly_wait(5) # неявное ожидание, на случай, если страница будет долго грузиться
    
    try: # заголовок вакансии
        topic = work_link.iloc[item].topic
            
    except:
        topic = 'none'
        
        
    try: # размер предлагаемой зп
        salary = (
            driver.find_element_by_class_name('resume-block-container')
            .find_element_by_tag_name('h2')
            .text
            .replace('\u2009', '')
            .replace(' руб.', '')
        )
    
    except:
        salary = 'none'
        
    
    try: # краткая информация о соискателе
        summary = driver.find_element_by_class_name('resume-header-title').text.split('\n')
        
    except:
        summary = 'none'
        
    try: # адрес
        full_resume = driver.find_element_by_class_name('resume-wrapper').text
        
    except:
        full_resume = 'none'
        
        
    data.append([topic, salary, summary, full_resume])

In [ ]:
# преобразовываю в датафрейм

head = ['topic', 'salary', 'summary', 'full_resume']
work = pd.DataFrame(data, columns=head)

In [ ]:
work.head()

In [ ]:
# Записываю в файл

work.to_excel('C:/00_Data/Клиники_вакансии/hh_resume_25_10_2021.xlsx', encoding='utf-8', index=False)

In [ ]:
driver.quit()